<a href="https://colab.research.google.com/github/Michal-Janulewski/Uczenie-Maszynowe-w-Finansach/blob/Quiz-2/Quiz_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
!pip install surprise
from surprise import SVD, KNNWithMeans
from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise.model_selection import cross_validate, GridSearchCV
from surprise import Dataset
from surprise import Reader

In [5]:
# Wczytanie danych
ratings = pd.read_csv('https://s3-us-west-2.amazonaws.com/recommender-tutorial/ratings.csv')
movies = pd.read_csv('https://s3-us-west-2.amazonaws.com/recommender-tutorial/movies.csv')

In [ ]:
# 1. Przegląd danych

# Liczba filmów Sci-Fi
sci_fi_movies = movies[movies['genres'].str.contains('Sci-Fi')]
num_sci_fi_movies = len(sci_fi_movies)
print("Liczba filmów Sci-Fi:", num_sci_fi_movies)

In [ ]:
# Rozkład ocen komedii
comedy_movies = movies[movies['genres'].str.contains('Comedy')]
comedy_ratings = ratings[ratings['movieId'].isin(comedy_movies['movieId'])]
comedy_ratings_distribution = comedy_ratings['rating'].value_counts().sort_index()
print("Rozkład ocen komedii:")
print(comedy_ratings_distribution)

In [ ]:
# Średnia ocen wszystkich filmów akcji oraz 3 filmy najwyżej oceniane
action_movies = movies[movies['genres'].str.contains('Action')]
action_ratings = ratings[ratings['movieId'].isin(action_movies['movieId'])]
average_action_rating = action_ratings['rating'].mean()

# Grupowanie ocen po identyfikatorze filmu, wyznaczenie średniej oceny i sortowanie malejąco
avg_ratings_per_movie = action_ratings.groupby('movieId')['rating'].mean().reset_index()
top_3_action_movies = avg_ratings_per_movie.nlargest(3, 'rating')

# Dołączenie tytułów filmów do 3 najwyżej ocenianych
top_3_action_movies_with_titles = pd.merge(top_3_action_movies, movies[['movieId', 'title']], on='movieId', how='left')

print("Średnia ocen wszystkich filmów akcji:", average_action_rating)
print("3 filmy najwyżej oceniane wśród filmów akcji:")
print(top_3_action_movies_with_titles[['title', 'rating']])

2.1. Zbuduj system na podstawie algorytmu SVD oraz kNNwithMeans


In [11]:
min_rating = ratings.rating.min()
max_rating = ratings.rating.max()

reader = Reader(rating_scale=(min_rating, max_rating))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)
algo = SVD()
model1=algo.fit(trainset)
predictions_svd = algo.test(testset)

In [ ]:
knn_model = KNNWithMeans()
model2= knn_model.fit(trainset)
predictions_knn = model2.test(testset)
knn_rmse = accuracy.rmse(predictions_knn)
print("RMSE dla modelu kNNwithMeans:", knn_rmse)

2.2. Czym różni się algorytm kNN with means od standardowego kNN?


In [ ]:
#Standardowy algorytm kNN przewiduje ocenę dla danego użytkownika lub elementu, znajdując k najbardziej podobnych sąsiadów i uśredniając ich oceny.
#Natomiast kNN with Means dodaje dodatkowo średnią ocenę użytkownika lub elementu do przewidywania oceny, co może poprawić jakość predykcji,
#szczególnie w przypadku niestabilnych danych. Innymi słowy, kNN with Means uwzględnia również ogólną tendencję ocen danego użytkownika lub elementu podczas przewidywania brakujących ocen.

2.3. Wykorzystaj metodę hiperparametryzacji GridSearch do wyboru liczby sąsiadów od 2-6

In [ ]:
param_grid = {'k': list(range(2, 7))}
knn_model = KNNWithMeans(sim_options={'name': 'cosine', 'user_based': False})
grid_search = GridSearchCV(KNNWithMeans, param_grid, measures=['RMSE'], cv=5)
model3= grid_search.fit(data)

print("Najlepsze RMSE:", grid_search.best_score['rmse'])
print("Najlepsze parametry:", grid_search.best_params['rmse'])

2.4. W ocenie metod wykorzystaj walidację krzyżową


In [ ]:
svd_cv_results = cross_validate(model1, data, measures=['RMSE', 'MAE'], cv=5, verbose=True);
knn_cv_results = cross_validate(model2, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

3. Podaj rekomendacje po obejrzeniu filmu: Jumanji  oraz Flint

In [ ]:
movies_to_watch = ["Jumanji", "Flint"]
def recommend_movies_knn(model, movies_to_watch, k=10):
    recommended_movies = []
    for movie_title in movies_to_watch:
        if movies['title'].str.contains(movie_title, case=False).any():
            movie_id = movies[movies['title'].str.contains(movie_title, case=False)]['movieId'].values[0]
            similar_movies = model.get_neighbors(movie_id, k=k)
            recommended_movies.extend(similar_movies)
        else:
            print(f"Nie można znaleźć filmu '{movie_title}' w zbiorze danych.")
    return recommended_movies

recommended_movies_knn = recommend_movies_knn(model2, movies_to_watch)

recommended_movie_titles = movies[movies['movieId'].isin(recommended_movies_knn)]['title'][:10]
movies_to_watch_titles = " i ".join(movies_to_watch)
print(f"Rekomendowane filmy po obejrzeniu filmów {movies_to_watch_titles}:")
for idx, title in enumerate(recommended_movie_titles, start=1):
    print(f"{idx}. {title}")